In [1]:
import os
import time
import pickle
import joblib
import warnings
from APIs import *
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from datetime import timedelta
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
os.environ['TZ'] = 'Asia/Calcutta'
pd.set_option('display.max_columns', 100)
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
pd.options.mode.chained_assignment = None

import pylab
pylab.rcParams['figure.figsize'] = (20, 6)

import xgboost as xgb
from sklearn.svm import SVR
from xgboost import plot_importance
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor

In [2]:
i = 25 # This is todays date 
raw_df = pickle.load(open('tpddl_data','rb'))
train_date  =  pd.to_datetime("2019-12-"+str(i-1))
prediction_date = pd.to_datetime("2019-12-"+str(i+1))
validation_date = pd.to_datetime("2019-12-"+str(i-15))
print(train_date, prediction_date, validation_date)

wdf_forecast = pickle.load(open("tpddl_weather_forecast_data","rb"))
wdf_forecast_data = wdf_forecast[(wdf_forecast.datetime.dt.date>=prediction_date-timedelta(1))&(wdf_forecast.datetime.dt.date<=prediction_date)]
wdf_forecast_data.head(1)

2019-12-24 00:00:00 2019-12-26 00:00:00 2019-12-10 00:00:00


,datetime,cloud_cover,apparent_temperature,temperature,humidity,dew_point,wind_speed,wind_bearing,precip_probability,precip_intensity
6175,2019-12-25,0.3,8.5,8.5,0.91,6.8,2.0,310.5,0.0,0.0


In [ ]:
raw_df

In [59]:
wdf = pickle.load(open("tpddl_weather_historical_all_data","rb"))
wdf = wdf[["datetime_local", 'apparent_temperature', 'temperature', 'humidity', 'dew_point', 
                 'wind_speed', 'cloud_cover', 'wind_bearing', 'precip_probability', 'precip_intensity']].copy()
weather_feature = ['apparent_temperature', 'temperature', 'humidity','dew_point', 'wind_speed',
                   'cloud_cover','wind_bearing','precip_probability', 'precip_intensity']

wdf.rename(columns= {'datetime_local': "datetime"},inplace=True)
wdf['datetime'] = pd.to_datetime(wdf['datetime'])
wdf.iloc[:,1:] = wdf.iloc[:,1:].astype("float64")
wdf.dropna(inplace=True)
wdf.drop_duplicates("datetime", inplace=True)
wdf = wdf[(wdf.datetime.dt.year <= 2019) & (wdf.datetime.dt.year >= 2015)]


# Outlier removing
wdf.loc[wdf['temperature'] < 0,   'temperature']          = np.nan
wdf.loc[wdf.temperature.isnull(), 'dew_point']            = np.nan
wdf.loc[wdf.temperature.isnull(), 'apparent_temperature'] = np.nan
wdf.loc[wdf.temperature.isnull(), 'humidity']    = np.nan
wdf.loc[wdf.temperature.isnull(), 'wind_speed']  = np.nan
wdf.loc[wdf.temperature.isnull(), 'cloud_cover'] = np.nan

wdf.loc[(wdf['wind_bearing'] > 359.0) & (wdf['wind_bearing'] < 0),  'wind_bearing'] = np.nan
wdf.loc[wdf.wind_bearing.isnull(), 'dew_point']   = np.nan
wdf.loc[wdf.wind_bearing.isnull(), 'apparent_temperature'] = np.nan
wdf.loc[wdf.wind_bearing.isnull(), 'humidity']    = np.nan
wdf.loc[wdf.wind_bearing.isnull(), 'wind_speed']  = np.nan
wdf.loc[wdf.wind_bearing.isnull(), 'cloud_cover'] = np.nan

wdf.loc[(wdf['wind_speed'] < 0) ,  'wind_speed'] = np.nan
wdf.loc[wdf.wind_speed.isnull(), 'dew_point']   = np.nan
wdf.loc[wdf.wind_speed.isnull(), 'apparent_temperature'] = np.nan
wdf.loc[wdf.wind_speed.isnull(), 'humidity']    = np.nan
wdf.loc[wdf.wind_speed.isnull(), 'wind_bearing']  = np.nan
wdf.loc[wdf.wind_speed.isnull(), 'cloud_cover'] = np.nan

wdf.set_index('datetime', inplace=True)
wdf.interpolate(method='time', inplace=True)
wdf = wdf.asfreq("15min")
wdf.interpolate(method='time', inplace=True)
wdf.reset_index(inplace=True)
wdf.drop_duplicates('datetime',inplace=True)
wdf.reset_index(inplace=True,drop=True)
wdf.drop(wdf[wdf.datetime.dt.date == pd.to_datetime("2016-02-29").date()].index,inplace = True)
wdf.shape # Shape will be 1,40,157   10

KeyError: "['datetime_local'] not in index"

In [ ]:
# Appending load and weather data
wdf = wdf[wdf.datetime.dt.date <= train_date]
wdf = wdf.append(wdf_forecast_data,sort=True)

df  = pickle.load(open("tpddl_data","rb"))
df = df[(df.datetime.dt.date <= train_date) & (df.datetime.dt.year >= 2015)]
df.drop(df[df.datetime.dt.date == pd.to_datetime("2016-02-29").date()].index,inplace = True)



In [ ]:
df = pd.merge(df, wdf, on='datetime', how='outer')

## Holidays removing
holidays = pd.read_excel("holidays.xlsx")
df = df[~df["date"].isin(pd.melt(holidays)["value"].dt.date)]
df.shape # 131136  13

In [ ]:
def plott(df):
    plt.figure(figsize=(20,6))
    plt.plot(df)
    plt.show()
    
def removing_outliers(df):
    df2 = df.copy()
    df2.drop(df2.loc[df2.datetime.dt.year == 2018,"apparent_temperature"].loc[81445:81478,].index, inplace = True)
    df2.drop(df2.loc[df2.datetime.dt.year == 2018,"apparent_temperature"].loc[81537:81571,].index, inplace = True)
    df2.drop(df2.loc[df2.loc[:,"dew_point"] <= 0,:].index, inplace = True)
    df2.reset_index(inplace=True, drop = True)
    return df2

def humidex(df):
        Td = df['dew_point'] + 273.15
        Tc = df['temperature']
        a = 1/273.15
        b = 1/Td
        c = Tc + 0.555*((6.11*(np.exp(5417.75*(a-b))))-10)
        return c
    
def calculate_RH( df):
        Es = 6.11*10.0**((7.5*df['temperature']) / (237.7 + df['temperature']))
        E = 6.11*10.0**((7.5*df['dew_point'])/(237.7 + df['dew_point']))
        RH = (E/Es) * 100
        return RH

def sinwave(df, feat, divs):
    return np.sin(df[str(feat)]*(2*np.pi/divs))

In [ ]:
def feature_engineering_load_realted(data_frame):
    
    df = data_frame.copy()
    # Adding time related features
    df['hour']   = df.datetime.dt.hour + 1   # Categorical
    df['date']   = df.datetime.dt.date       # Dates dont consider this    ( No repeatation )
    df['dom' ]   = df.datetime.dt.day        # Categorical 30
    df['month']  = df.datetime.dt.month      # Categorical 12
    df['year']   = df.datetime.dt.year       # Categorical dont consider this 12
    df['dow' ]   = df.datetime.dt.dayofweek  # Categorical 6
    df['doy' ]   = df.datetime.dt.dayofyear  # Categorical may be consider ( No repeatation )
    df['tb']     = df.datetime.apply(lambda x : ((x.hour*60 + x.minute)//15+1)) # Time blocks 96
    df['weekend']= np.where(df['dow']>5, 1,0)  # Imbalanced category

    # This feature is add
    df['woy']    = df.datetime.dt.weekofyear # Categorical 52
    
    df['lag2h'] = df.load.shift(8)
    df['lag4h'] = df.load.shift(16)
    
    df['load_ewm1'] = df['load'].ewm(span = 6).mean()
    df['load_ewm2'] = df['load'].ewm(span =10).mean()
    
    df['sin_doy']   = sinwave(df, "doy" , 365)
    df['sin_tb']    = sinwave(df, "tb"  ,  96)
    df['sin_dow']   = sinwave(df, "dow" ,   7)
    df['sin_month'] = sinwave(df, "month", 12)
    df["sin_woy"]   = sinwave(df, "woy"  , 52) # Adding
    
    hour_mean = df.groupby(['date','hour']).mean()["load"].reset_index().rename(columns = {'load':'hour_mean'})
    df = pd.merge(df, hour_mean, on=['date','hour'], how='left')

    return df

In [ ]:
def feature_engineering_weather_realted(data_frame):
    span = 32

    df = data_frame.copy()
    df['humidity'] = ((6.11*10.0**(7.5*df.dew_point/(237.7+df.dew_point)))*100)/((df.temperature + 273.15)*461.5)
    df['humidex'] = humidex(df)
    df['RH'] = calculate_RH(df)

    df['tb_aptemp']   = df.tb   * df.apparent_temperature
    df['tb_load']     = df.tb   * df.load
    df['load_aptemp'] = df.load * df.apparent_temperature
    
    df['aptemp_ewm_6h']  = df['apparent_temperature'].ewm(span=24).mean()
    df['aptemp_ewm']     = df['apparent_temperature'].ewm(span=span).mean()
    df['temperature_ewm'] = df['temperature'].ewm(span=span).mean()
    df['dew_point_ewm']   = df['dew_point'].ewm(span=span).mean()
    df['humidity_ewm']    = df['humidity'].ewm(span=span).mean()
    df['wind_speed_ewm']  = df['wind_speed'].ewm(span=span).mean()
    df['cloud_cover_ewm'] = df['cloud_cover'].ewm(span=span).mean()
    df['wind_bearing_ewm']       = df['wind_bearing'].ewm(span=span).mean()
    df['precip_intensity_ewm']   = df['precip_intensity'].ewm(span=span).mean()
    df['precip_probability_ewm'] = df['precip_probability'].ewm(span=span).mean()
    df['load_ewm']             = df['load'].ewm(span=span).mean()
    
    
    df['load_aptemp_ewm6']  = df['load_aptemp'].ewm(span=24).mean()
    
    df['wt']=df.dow*df.hour
    df['tm']=df.temperature*df.month
    df['tm2']=(df.temperature**2)
    df['th']=df.temperature*df.hour
    df['th2']=(df.temperature**2)*df.hour
    
    return df

In [ ]:
#df = removing_outliers(df)
df = feature_engineering_load_realted(df)
df = feature_engineering_weather_realted(df)

In [ ]:
df.tail()

In [ ]:
weather_feature = ['apparent_temperature','temperature','humidity',#'dew_point',
'wind_speed','cloud_cover','wind_bearing','precip_probability','precip_intensity',
'humidex','RH','tb_aptemp',#'load_aptemp','tb_load',
'aptemp_ewm_6h','aptemp_ewm','temperature_ewm','dew_point_ewm','humidity_ewm','wind_speed_ewm','cloud_cover_ewm',
'wind_bearing_ewm','precip_intensity_ewm','precip_probability_ewm',
#'sdtbrm','sdtbrm2','3tbrm','3tbrmw',
'load_aptemp_ewm6','load_aptemp_ewm12','load_aptemp_ewm24']

'dew_point' === 'humidity'              0.99   # Take humidity
'dew_point_ewm' === 'humidity_ewm'      0.99   #
'humidity' === 'humidity_ewm'           0.99
'dew_point' === 'dew_point_ewm'         0.99
'aptemp_ewm_6h' === 'aptemp_ewm'        0.99
'aptep_ewm_6h' === 'aptemp'             0.97
'load_aptemp_12h' === 'load_aptemp_24h' 0.99
'load_aptemp_6h' === 'load_aptemp_12h'  0.99

'load_aptemp_12' === "load_aptemp6" == 0.99
'load_aptemp_6' === 'load_aptemp' ==0.96

In [ ]:
plt.figure(figsize=(22,18))
sns.heatmap(df[weather_feature].corr(),square=True,annot=True,cmap="YlGnBu")
plt.show()

# Shifting features

In [ ]:
features = ['datetime','load', 'lag2h','lag4h', 'sin_woy','sin_tb','sin_doy', 'dow','doy','month','hour',
'apparent_temperature','humidity','wind_speed','cloud_cover','temperature','RH','humidex',
'hour_mean',  'aptemp_ewm','load_ewm','wind_speed_ewm','cloud_cover_ewm',
'load_aptemp','wt','tm','tm2',
#'wind_bearing_ewm','precip_intensity_ewm','precip_probability_ewm' # Adding
           ]

shift_features = ['apparent_temperature','humidity','wind_speed','cloud_cover','temperature','RH','humidex',
                  'aptemp_ewm','wind_speed_ewm','cloud_cover_ewm',
                  'wind_bearing_ewm','precip_intensity_ewm','precip_probability_ewm', # Adding
                 'tb_aptemp','doy','month','dow', 'sin_doy','sin_month','sin_woy','wt','tm','tm2']

features  ['datetime','load',    'lag1','lag2','lag3','lag5',    'sin_tb','sin_doy','sin_woy'
           'dow','doy','month','year','hour',
           'apparent_temperature','humidity','wind_speed','cloud_cover','temperature',"dew_point","RH",'humidex'
           'hour_mean',
           
           'aptemp_mean_12h','aptemp_mean_6h'
        ,'tb_aptemp','tb_load'
        'load_aptemp', 'load_ewm' #'load_aptemp_ewm12','load_aptemp_ewm6',

           #'temperature_ewm'
           'tm4','hm12','dp_ewm','3tbrmw','lagcwm','wsp_12','cc_12',\
            'wt','tm','tm2','tm3','']


shift_features  ['temperature','apparent_temperature','humidex','wind_speed','cloud_cover','dew_point','RH','humidity'
                 'temperature_ewm','dp_ewm','aptemp_mean_6h','aptemp_mean_12h',
                 'dow','doy','month',
                 'tb_aptemp',,'hm12','
                ,'wsp_12','cc_12','wci','wt','tm','tm2','tm3','tm4']
categorical_features  ['dow','month','year','hour']

In [ ]:
for i in shift_features:
    df[i] = df[i].shift(-192)

df = df[features].copy()
df.dropna(inplace=True)
df['target'] = df.load.shift(-192)


validation = 15 * 96

x_train, y_train   = df[df.target.notna()].iloc[:-validation,1:-1], df[df.target.notna()].iloc[:-validation,-1]
x_valid, y_valid   = df[df.target.notna()].iloc[-validation:,1:-1], df[df.target.notna()].iloc[-validation:,-1] 
x_test = df.iloc[-96:].iloc[:,1:-1]

# Hyper-parameters Tuning.

joblib.dump(model,path)
model = joblib.load(path)

In [ ]:
# First we set default parameters like this
params = { "learning_rate" : 0.02,
         "n_estimators" : 1200,
         "max_depth" : 9,        #Tuned
         "min_child_weight" : 3, #Tuned
         "gamma" : 0.5,          #Tuned
         "subsample" : 0.6,      #Tuned
         "colsample_bytree" : 0.5, # Tuned
         "objective" : "reg:squarederror",
         "verbosity" : 1
         }

xgb_model = xgb.XGBRegressor(**params)

dm_train = xgb.DMatrix(x_train,y_train)
dm_valid = xgb.DMatrix(x_valid,y_valid)
dm_test  = xgb.DMatrix(x_test)

In [ ]:
xgb_model = xgb.train(params, dm_train)

In [ ]:
## These two used to see eval_set error just pass both of this into fit method
eval_set  = [ (x_train, y_train) , (x_valid, y_valid)]
eval_metric = ["rmse","mae"]

# Training start of XGBoost
xgb_model.fit(x_train, y_train,
        #eval_set = eval_set,
        #eval_metric = eval_metric,
        verbose=True)

#evals_result = xgb_model.evals_result()

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return (np.sum(np.abs(y_true - y_pred)/y_true)*100) / len(y_pred)

def mape_error(y_pred, dm_train):
    y_true = dm_train.get_label()
    return "mape_error" , float((np.sum(np.abs(y_true - y_pred)/y_true)*100) / len(y_pred))

# pass xgb.train(feval=mape_error)

In [ ]:
# It shows the validation vs training MAE error with respect to n_estimators
plt.figure(figsize = (20,6))
pd.DataFrame(evals_result['validation_0']).mae.plot(label="train")
pd.DataFrame(evals_result['validation_1']).mae.plot(label="valid")
plt.legend()
plt.show()

In [ ]:
param_dist = {learning_rate = 0.1,
              n_estimators = 1000,
              max_depth = 9,           #Tuned
              min_child_weight = 3,    #Tuned
              gamma = 0.5,             #Tuned
              subsample = 0.6,         #Tuned
              colsample_bytree = 0.5,  #Tuned
              objective = "reg:squarederror",
              verbosity = 1
}

eval_set  = [ (x_train, y_train) , (x_valid, y_valid)]
eval_metric = mape_error

clf = xgb.XGBModel(**param_dist)
clf.fit(x_train, y_train,
        eval_set = eval_set,
        eval_metric = eval_metric,
        verbose=False)

evals_result = clf.evals_result()

In [ ]:
xgb.train({'tree_method': 'hist', 'seed': 1994,
           'disable_default_eval_metric': 1},
          dtrain=dtrain,
          num_boost_round=10,
          obj=squared_log,
          feval=rmsle,
          evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
          evals_result=results)

In [ ]:
y_true = raw_df[raw_df.datetime.dt.date == prediction_date.date()].load.values
y_pred = xgb_model.predict(x_test)
plt.figure(figsize=(20,6))
plt.plot(y_true,label="actual")
plt.plot(y_pred,label="predict")
plt.legend()
plt.show()

In [ ]:
mape_store = pickle.load(open("mape_store",'rb'))

In [ ]:
mape_store

In [ ]:
mean_absolute_percentage_error(y_true,y_pred)

In [ ]:
mean_absolute_percentage_error(y_true,y_pred)

In [ ]:
mape_store.append((prediction_date.date(),mean_absolute_percentage_error(y_true,y_pred)))

In [ ]:
results = pd.DataFrame({"datetime": np.array(mape_store)[:,0], "mape_new":np.array(mape_store)[:,1]})
results.sort_values("datetime",inplace=True)

In [ ]:
mape_store.append(((prediction_date + timedelta(2)).date(), 5.3449075403589275))

In [ ]:
pickle.dump(mape_store,open("mape_store","wb"))

In [ ]:
2761

In [ ]:

# If you get 0.8 for both of the params then try this
'subsample':[0.75, 0.8, 0.85],
'colsample_bytree':[0.75, 0.8, 0.85]
    
# Try to tune regularizations 
'reg_alpha':[0.0005, 0.02, 0.1, 1, 100]
    
# We got optimum 0.1
'reg_alpha':[0.001, 0.005, 0.01, 0.05]    

In [ ]:
def modelfit(model, x_train, y_train, useTrainCV = True, cv_folds = 5, early_stopping_rounds = 20 ):
    
    if useTrainCV:
        # This gives us a good start of n_estimators
        print("Before Test n_estimator: ",model.get_xgb_params()['n_estimators'])
        xgb_param = model.get_xgb_params()
        dm_train = xgb.DMatrix(x_train,y_train)
        cvresult = xgb.cv(xgb_param, dm_train,num_boost_round = xgb_model.get_params()['n_estimators'],
                  nfold = cv_folds,
                  metrics= ["rmse","mae"],
                  early_stopping_rounds = 5)
        
        print("After Test n_estimator: ",cvresult.shape[0])
        model.set_params(n_estimators=cvresult.shape[0])
    
    # This gives us a score after training
    print("Model fitting starts...")
    eval_set  = [ (x_train, y_train) , (x_valid, y_valid)]
    eval_metric = ["rmse","mae"] 
    model.fit(x_train, y_train, 
              eval_metric=eval_metric, 
              eval_set = eval_set)
    
    return model
    #Predict training set:
    y_pred = model.predict(y_train.values)
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print "Accuracy : %.4g" % metrics.accuracy_score(y_train.values, y_pred)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
#tree_method='gpu_hist', gpu_id=0


params = {'n_estimators':500,
          'objective':'reg:squarederror',
          'booster':'gbtree',
          'max_depth':50,
          'learning_rate':0.2,
          'colsample_bytree':0.6,
          'colsample_bylevel':0.6,
          'subsample':0.8,
          'min_child_weight':1,
          'gamma':0.5}
            
model = xgb.train(params, dm_train, num_boost_round=2000, evals=[(dm_test, "Test")], early_stopping_rounds=15)

In [ ]:
# Inference of Prediction date

dm_predict = xgb.DMatrix(X_predict)
y_pred = model.predict(dm_predict)

# Getting actual load values
y_actual = np.array(raw_df[raw_df.datetime.dt.date == prediction_date].load)

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
plot_importance(model, max_num_features=30, height=0.8, ax=ax)
ax.grid(False)
plt.title("XGBoost - Feature Importance", fontsize=15)
plt.savefig("feature_imp.jpg")
plt.show()

In [ ]:
mape = (sum((y_actual - y_pred)/y_actual)*100)/96
mape

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(y_pred,label='Pred')
plt.plot(y_actual,label='Actual')
plt.legend()
plt.show()

In [ ]:
xgb_clf = XGBRegressor( objective = 'reg:squarederror')

params = {"n_estimators" : [300,350,400,500,600,700,900],
 "learning_rate"    : [0.02, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30] ,
 "max_depth"        : [ 8, 10, 12, 15, 18, 30, 40],
 "min_child_weight" : [ 1, 3, 5, 7],
 "gamma"            : [ 0.2, 0.4, 0.6, 0.8, 1, 1.5],
 "colsample_bytree" : [ 0.3, 0.4, 0.5, 0.6, 0.7,0.8,1 ],
 "subsample"        : [0.2, 0.4, 0.5, 0.6, 0.7],
  "reg_alpha"       : [0, 0.5, 1],
  "reg_lambda"      : [1, 1.5, 2, 3, 4.5],
}


#'colsample_bylevel' this is not present


xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = params,
                             cv = 7, verbose = 3)

model_xgb = xgb_rscv.fit(X_train,y_train)

print("Best: %f using %s" % (model_xgb.best_score_, model_xgb.best_params_))

print(model_xgb.score(x_test,y_test))

In [ ]:
signal.savgol_filter(ea['final']['forecast'], 25, 5)

In [ ]:
param_test_1 = {
 'gamma':[0.1,0.2, 0.3, 0.4, 0.5],
 'max_depth':[9,10,11]
}
xgb_reg = RandomizedSearchCV(xgb_model, param_test_1, n_iter=10, verbose=2, cv=5, iid=True)

print("Randomized search..")
search_time_start = time.time()
xgb_reg.fit(x_train, y_train)
print("Randomized search time:", time.time() - search_time_start)

In [ ]:
# First we set default parameters like this
xgb_model = xgb.XGBRegressor(learning_rate = 0.1,
                            n_estimators = 100,
                            max_depth = 9,
                            min_child_weight = 3,
                            gamma = 0.5,
                            subsample = 0.8,
                            colsample_bytree = 0.8,
                            objective = "reg:squarederror",
                            verbosity = 1
                            )

## Backcasting comparisions..

In [ ]:
from DBinteractor import DBInteractor
from functions import functions
db_interactor = DBInteractor()
fun = functions()

In [ ]:
date_from, date_to = fun.get_date(-44), fun.get_date(-30)
date_from, date_to

In [ ]:
forecast_live = db_interactor.get_forecast_live(30,date_from,date_to)

In [ ]:
forecast_vrx  = db_interactor.get_forecast(30,date_from,date_to,'vrx')
forecast_vrl  = db_interactor.get_forecast(30,date_from,date_to,'vrl')
forecast_live = db_interactor.get_forecast_live(30,date_from,date_to)
forecast_all  = pd.DataFrame({"datetime": forecast_live.datetime,
            "vrx":forecast_vrx.forecast,"vrl":forecast_vrl.forecast,
             'live':forecast_live.forecast})

In [ ]:
#forecast_vvn = db_interactor.get_forecast(75,date_from,date_to,'vvn')

In [ ]:
forecast_all["actual"] =  raw_df[(raw_df.datetime.dt.date >= pd.to_datetime(date_from)) & 
       (raw_df.datetime.dt.date <= pd.to_datetime(date_to))].load.reset_index(drop=True)

In [ ]:
len(forecast_vrx), len(forecast_vvn), len(forecast_vrl), len(forecast_live)

In [ ]:
forecast_all

In [ ]:
forecast_all

In [ ]:
results.reset_index(drop=True,inplace=True)

In [ ]:
results['datetime']

In [ ]:
results['vrx'] = None
results['vrl'] = None
results['vlive'] = None

for i in range(17,32):
    date = pd.to_datetime("2019-12-" + str(i)).date()
    dataa = forecast_all[forecast_all.datetime.dt.date == date]
    results['vrx'][results.datetime == date] = mean_absolute_percentage_error(dataa.actual, dataa.vrx) 
    results['vrl'][results.datetime == date] = mean_absolute_percentage_error(dataa.actual, dataa.vrl)
    results['vlive'][results.datetime == date] = mean_absolute_percentage_error(dataa.actual, dataa.live)    

In [ ]:
results.index = range(17,32)
plt.figure(figsize=(20,10))
plt.plot(results.mape_new,label="NEW")
plt.plot(results.vrx,label="VRX")
plt.plot(results.vrl,label="VRL")
plt.plot(results.vlive,label="VLIVE")
plt.xticks(range(17,32))
plt.xlabel("December 2019")
plt.legend()
plt.show()